In [1]:
import ee
import geepy
import math
import sys

ee.Initialize()

In [2]:
config = geepy.params.configParams('input_classification_v4.json')

amostras = ee.FeatureCollection(config.params['samples'])

watersheds = ee.FeatureCollection(config.params['studyArea']) 

pivots = {i: ee.FeatureCollection('users/fernandompimenta/AIBA/centralPivots/pc' + i) for (i) in config.params['years2process']}

bands = config.params['bandParams']

l5 = ee.ImageCollection(config.params['imgCollection']['lc5']['id']).select(
    config.params["imgCollection"]["lc5"]["bands"],
    config.params["imgCollection"]["lc5"]["bandNames"])

l7 = ee.ImageCollection(config.params['imgCollection']['lc7']['id']).select(
    config.params["imgCollection"]["lc7"]["bands"],
    config.params["imgCollection"]["lc7"]["bandNames"])

l8 = ee.ImageCollection(config.params['imgCollection']['lc8']['id']).select(
    config.params["imgCollection"]["lc8"]["bands"],
    config.params["imgCollection"]["lc8"]["bandNames"])

In [3]:
# Modelo digital de elevação
srtm = ee.Image(config.params['srtm'])

# Localities distace
villages = ee.FeatureCollection(config.params['villages'])
towns = ee.FeatureCollection(config.params['towns'])
farms = ee.FeatureCollection(config.params['farms'])
hamlets = ee.FeatureCollection(config.params['hamlets'])
rivers = ee.FeatureCollection(config.params['rivers'])

dvillage = villages.distance(config.params['radist'])
dfarm = farms.distance(config.params['radist'])
dtown = towns.distance(config.params['radist'])
dhamlet = hamlets.distance(config.params['radist'])
driver = rivers.distance(config.params['radist'])

slope = ee.Terrain.slope(srtm)
aspect = ee.Terrain.aspect(srtm).divide(180).multiply(math.pi).sin()
hillshade = ee.Terrain.hillshade(srtm)

ntl30m = {}
for i in config.params['years2process']:
    viirs = ee.Image(config.params['VIIRS'][i]).select('avg_rad').divide(100)
    ntl30m[i] = viirs.resample('bilinear').reproject(
        crs = viirs.projection().crs(),
        scale = 30
    )

In [4]:
landsat = {}

for year in config.params['years2process']:  
    start_d = year + config.params['period']['dry']['start']
    end_d = year + config.params['period']['dry']['end']
    #print("Start-End dry season %s %s" %(start_d, end_d))

    if(int(year) < 2002):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d)
    elif(int(year) in (2002,2011,2012)):
        filtered = l7.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d)
    elif(int(year) > 2002 and int(year) < 2011):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d)
    else:
        filtered = l8.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d)

    fEdgeRemoved = filtered.map(geepy.image.edgeRemoval).median()
    
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, ntl30m[year], 'ntl')
    
    fEdgeRemoved = geepy.image.calcNDBI(fEdgeRemoved)

    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, srtm, 'srtm')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, slope, 'slope')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, aspect, 'aspect')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, hillshade, 'hillshade')
    
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, dvillage, 'dvillage')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, dfarm, 'dfarm')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, dtown, 'dtown')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, dhamlet, 'dhamlet')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, driver, 'driver')
    
    fEdgeRemoved = geepy.image.calcNDVI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcEVI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcSAVI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcNDWI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcMNDWI(fEdgeRemoved)

    fEdgeRemoved = geepy.image.calcFractions(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcNDFI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcNDFI2(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcNDFI3(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcFCI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcGVNPV(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcNPVSOIL(fEdgeRemoved)

    ndvithermal = fEdgeRemoved.select('ndvi').divide(fEdgeRemoved.select('thermal'))
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, ndvithermal, 'ndvithermal')
    
    landsat[year] = fEdgeRemoved.clip(watersheds)
    
    sys.stdout.write("\rProcessing Landsat data: %s" % year)
    sys.stdout.flush()

Processing Landsat data: 2016

In [5]:
amostragem = geepy.feature.vec2rast(amostras, 'CLASS')


n = 2000
classBand = 'CLASS'
cv = [1,2,3,4,5,6,7]
cp = [n for i in range(len(cv))]

samples = geepy.image.randomSamples(amostras, amostragem, n, 369, classBand, cv, cp)
cp[6] = 1000
# 1 Formações florestais --- green
# 2 Formações savânicas ---- lightgreen
# 3 Agricultura Sequeiro --- magenta
# 4 Agricultura Irrigada --- pink
# 5 Pastagem --------------- yellow
# 6 Corpos d'água ---------- blue
# 7 Área urbana ------------ red

In [12]:
landsat['2016'].bandNames().getInfo()

['blue',
 'green',
 'red',
 'nir',
 'swir1',
 'swir2',
 'thermal',
 'ntl',
 'ndbi',
 'srtm',
 'slope',
 'aspect',
 'hillshade',
 'dvillage',
 'dfarm',
 'dtown',
 'dhamlet',
 'driver',
 'ndvi',
 'evi',
 'savi',
 'ndwi',
 'mndwi',
 'gv',
 'npv',
 'soil',
 'cloud',
 'gvs',
 'shade',
 'ndfi',
 'ndfi2',
 'ndfi3',
 'fci',
 'gvnpv',
 'npvsoil',
 'ndvithermal']

In [6]:
for year in config.params['years2process']:
    pivots[year] = geepy.feature.vec2rast(pivots[year], 'CLASS').reproject(
        crs = landsat[year].select('nir').projection().crs(),
        scale = 30
    )
    sys.stdout.write("\rProcessing Central Pivots data: %s" % year)
    sys.stdout.flush()

Processing Central Pivots data: 2016

In [7]:
training = geepy.image.trainingSamples(landsat['2016'], samples)

In [8]:
classification2016 = geepy.image.randomForest(landsat['2016'], training, bands)

In [9]:
classification = {year: geepy.image.randomForest(landsat[year], training, bands) for (year) in config.params['years2process']}

In [10]:
classRemapped = {year: classification[year].remap([1,2,3,4,5,6,7],[1,2,3,3,5,6,7]).rename('classification'+year) for (year) in config.params['years2process']}

AttributeError: 'tuple' object has no attribute 'remap'

In [ ]:
finalClassification = {year: classRemapped[year].where(pivots[year].select('CLASS'), 4) for year in config.params['years2process']}

In [ ]:
mapa = geepy.maps.geeMap(watersheds, zoom=8)

# Training

# 1 Flormações florestais -------------------------------- green
# 2 Formações savânicas ---------------------------------- lightgreen
# 3 Agricultura Sequeiro --------------------------------- magenta
# 4 Agricultura Irrigada --------------------------------- pink
# 5 Pastagem --------------------------------------------- yellow
# 6 Corpos d'água ---------------------------------------- blue
# 7 Área urbana/Construções rurais ----------------------- red

viz_params = {'min':1,'max':7,'palette':'green, lightgreen, magenta, pink, yellow, blue, red','format':'png'}

l5_viz_params = {'min':0,'max':0.5,'bands':'swir1,nir,red','format':'png'}

mapa.addLayer(landsat['1992'], viz_params=l5_viz_params, name='landsat')
mapa.addLayer(finalClassification['1992'], viz_params=viz_params, name='classificação')
#mapa.addLayer(amostragem.select('CLASS'), viz_params = viz_params, name = 'amostragem')

#mapa.addLayer(samples, name='pontos')

mapa.addControls()

In [ ]:
coords = [[[-46.632916356914535, -15.447083782725066],\
   [-43.13041651144095, -15.447083782725066],\
   [-43.13041651144095, -10.181249376641395],\
   [-46.632916356914535, -10.181249376641395],\
   [-46.632916356914535, -15.447083782725066]]]

In [ ]:
# To Asset
# tasks = {year: geepy.image.send2asset(finalClassification[year], coords, 'classification'+year+'v4', 'users/fernandompimenta/AIBA/classification/classification_'+year+'_v4', 30) for year in config.params['years2process']}          

In [ ]:
# To drive
# tasks = {year: geepy.image.send2drive(finalClassification[year], coords, 'classification'+year+'v4', 'classification_v4', 30) for year in config.params['years2process']}          